In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import ImageFolder
import os
from PIL import Image
import numpy as np
import tqdm as tqdm

In [4]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, 3, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        
        # Decoder layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(1),
            nn.Sigmoid()  # Output pixel values between 0 and 1
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
def transform_images_to_tensors(path):
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    files = os.listdir(path)
    images = []
    for i in files:
        img_tensor = transform((Image.open(path + i)).convert('L'))
        images.append(img_tensor)

    return images

In [ ]:
def transform_sketches_to_tensors(path):
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    files = os.listdir(path)
    images = []
    for i in files:
        img_tensor = transform(Image.open(path + i))
        images.append(img_tensor)
    return images

In [ ]:
# Load the image dataset and their corresponding sketches
image_dataset = transform_images_to_tensors('E:/ANN_PROJECT/ALL_Images/')

for i in range(0, 10000, 32):
    smaller_tensor1 = image_dataset[i : i+32]
    torch.stack(smaller_tensor1)
    filename1 = "ImageTensor"+str(i//32)+".pt"
    torch.save(smaller_tensor1, "E:/ANN_PROJECT/ImageTensors/" + filename1)

del image_dataset

In [ ]:
sketch_dataset = transform_sketches_to_tensors('E:/ANN_PROJECT/EqualizedSketches/')

for i in range(0, 10000, 32):
    smaller_tensor2 = sketch_dataset[i : i+32]
    torch.stack(smaller_tensor2)
    filename2 = "SketchTensor"+str(i//32)+".pt"
    torch.save(smaller_tensor2, "E:/ANN_PROJECT/SketchTensors/" + filename2)

del sketch_dataset

MAKING THE MODEL

In [ ]:
model = Autoencoder().cuda()

optimizer = optim.Adam(model.parameters(), lr=1e-3 , weight_decay=1e-5)
criterion = nn.MSELoss()

TRAINING THE MODEL

In [ ]:
x_data_dir = "E:/ANN_PROJECT/ImageTensors/"
y_data_dir = "E:/ANN_PROJECT/SketchTensors/"
num_epochs = 30

x_data_files = [f for f in os.listdir(x_data_dir) if f.endswith(".pt")]
y_data_files = [f for f in os.listdir(y_data_dir) if f.endswith(".pt")]
# loop through each x_train file for 30 epochs
for epoch in tqdm(range(num_epochs)):
    running_loss = 0
    for j in range(len(x_data_files) - 1):
        x_data = torch.load(os.path.join(x_data_dir, x_data_files[j]))
        x_data = torch.stack(x_data)

        y_data = torch.load(os.path.join(y_data_dir, y_data_files[j]))
        y_data = torch.stack(y_data)

        data = TensorDataset(x_data, y_data)
        data_loader = DataLoader(data, batch_size=32, shuffle=True)

        
        for i,data in enumerate(data_loader):
            img, sketch = data

            img = img.to('cuda')
            sketch = sketch.to('cuda')

            optimizer.zero_grad()
            output = model(img)
            loss = criterion(output, sketch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            

        del x_data
        del y_data


    # print training progress after each epoch
    print("EPOCH : " , epoch+1 , "LOSS : " , running_loss)

In [ ]:
# Load the model architecture
model = Autoencoder()  # Replace with your model architecture

model_path = 'model.pth'
model = torch.load(model_path , map_location=torch.device('cpu'))

TESTING PART

In [33]:
x_data_dir = "E:/ANN_PROJECT/ImageTensors/"
y_data_dir = "E:/ANN_PROJECT/SketchTensors/"

x_data_files = [f for f in os.listdir(x_data_dir) if f.endswith(".pt")]

x_data = torch.load(os.path.join(x_data_dir , x_data_files[-1]))

x_data = torch.stack(x_data)

# print(x_data.shape)
# print(len(x_data))
for i in range(len(x_data)):
    output = model(x_data[i].unsqueeze(0))
    torchvision.utils.save_image(output, 'E:/ANN_PROJECT/Predicted_Sketches/Sketch'+str(i)+'.png')

In [2]:
def transform_test_images_to_tensor(path):
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor()
    ])
    
    files = os.listdir(path)
    images = []
    for i in files:
        img_tensor = transform((Image.open(path + i)).convert('L'))
        images.append(img_tensor)
    return images

In [8]:
import os
from PIL import Image

# Directory path containing the images
directory = 'TestImage/'

image_shapes = []
# Iterate over each file in the directory
for filename in os.listdir(directory):
    # Construct the full file path
    file_path = os.path.join(directory, filename)
    
    # Open the image using PIL
    image = Image.open(file_path)
    
    # Get the shape of the image
    image_shapes.append(image.size)



images = transform_test_images_to_tensor('TestImage/')
images = torch.stack(images)
print(images.shape)

model = Autoencoder()  # Replace with your model architecture

model_path = 'model.pth'
model = torch.load(model_path , map_location=torch.device('cpu'))

output = model(images)
for i in range(len(output)):
    # resized_image = F.interpolate(output[i].unsqueeze(0), size=image_shapes[i], mode='bilinear', align_corners=False)
    torchvision.utils.save_image(output[i], 'TestSketch/Sketch'+str(i)+'.png')

torch.Size([4, 1, 512, 512])


In [7]:
from PIL import Image

# Open the two images
image1 = Image.open("TestImage/MadaraUchiha.jpg")
image2 = Image.open("TestSketch/Sketch2.png")

# Resize the images to have the same height
height = min(image1.height, image2.height)
image1 = image1.resize((int(image1.width * height / image1.height), height))
image2 = image2.resize((int(image2.width * height / image2.height), height))

# Create a new image with double the width
new_width = image1.width + image2.width
new_image = Image.new("RGB", (new_width, height))

# Paste the two images side by side
new_image.paste(image1, (0, 0))
new_image.paste(image2, (image1.width, 0))

# Save the combined image
new_image.save("combined_image4.jpg")